<a href="https://colab.research.google.com/github/JiaWeiXie/ntub_ai_bi/blob/main/ntub_bi_20220517.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz \
  && sudo tar -xzf ta-lib-0.4.0-src.tar.gz \
  && sudo rm ta-lib-0.4.0-src.tar.gz \
  && cd ta-lib/ \
  && sudo ./configure --prefix=/usr \
  && sudo make \
  && sudo make install \
  && cd ~ \
  && sudo rm -rf ta-lib/ \
  && pip install -U yfinance backtrader ta-lib

     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 6.4 MB 57.5 MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.24-cp37-cp37m-linux_x86_64.whl size=1465730 sha256=73d57c1de41064504be24a8714af095aa8b746d7d9a3261b70ad9ab4978f55b8
  Stored in directory: /root/.cache/pip/wheels/02/b1/a5/aca31e8cdd0137d7a83bf72237056b8705663dd1f9b5eac11e
Successfully built ta-lib
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascien

In [2]:
! pip install rich

     |████████████████████████████████| 231 kB 4.3 MB/s 
     |████████████████████████████████| 51 kB 6.3 MB/s 


In [3]:
import os
import sys
import math
import datetime

import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.analyzers as btanalyzers

from rich import print

In [4]:
import yfinance as yf

spy = yf.Ticker("SPY")
dataset = spy.history(period="max")
# dataset.to_csv("spy_max.csv")
dataset["2019"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,233.141438,238.098477,233.113005,237.122223,126925200,0.0,0
2019-01-03,235.273977,235.596243,230.951983,231.463806,144140700,0.0,0
2019-01-04,234.667414,239.899310,234.269337,239.216888,142628800,0.0,0
2019-01-07,239.501217,242.591061,238.553410,241.103012,103139100,0.0,0
2019-01-08,243.415679,243.880094,240.742857,243.368271,102512600,0.0,0
...,...,...,...,...,...,...,...
2019-12-24,310.460959,310.509234,309.910472,310.229187,20270000,0.0,0
2019-12-26,310.634754,311.890252,310.625116,311.880585,30911200,0.0,0
2019-12-27,312.653182,312.711125,311.243190,311.803314,42528800,0.0,0


In [144]:
class AllIn(bt.Sizer):
    def _getsizing(self, comminfo, cash, data, isbuy):
        # print(data[0], data[1], data.open[0], data.open[1])
        price = data.open[1]
        # print(f"${price:.2f}")
        position = self.broker.getposition(data)
        if isbuy:
            size = cash / price
        else:
            size = position.size

        return size

In [141]:
class SMACross(bt.Strategy):
    def log(self, txt, dt=None):
        if not self.p.printlog:
            return

        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt}: {txt}", end="\n\n")
    
    params = dict(
        ma_period_short=5,
        ma_period_long=10,
        printlog=True,
    )

    def __init__(self):
        # sma_5 = bt.ind.SimpleMovingAverage(
        #     self.data.close,
        #     period=self.p.ma_period_short,
        #   )
        # sma_10 = bt.ind.SimpleMovingAverage(
        #     self.data.close,
        #     period=self.p.ma_period_long,
        #   )
        # # self.ma_diff = sma_5 - sma_10

        sma1 = bt.ind.SMA(period=self.p.ma_period_short)
        sma2 = bt.ind.SMA(period=self.p.ma_period_long)
        self.crossover = bt.ind.CrossOver(sma1, sma2)

        self.dataopen = self.datas[0].open

    def _log_tx(self, crossover, price, txdate=None, isbuy=False):
        crossover = int(crossover)
        txdate = txdate if txdate else self.datas[0].datetime.date(0)
        if isbuy:
            self.log(
                f"[bold green]buy[/], {crossover}, ${price:.2f}",
                txdate,
            )
        else:
            self.log(
                f"[bold red]sell[/], {crossover}, ${price:.2f}",
                txdate,
            )


    def next(self):
        if len(self.data) == self.data.buflen():
            if self.position:
                self.close(price=self.dataopen[0])
                self._log_tx(self.crossover[0], self.dataopen[0])

            return

        if not self.position and self.crossover[0] > 0:
            self.buy(price=self.dataopen[1])
            self._log_tx(
                self.crossover[0],
                self.dataopen[1],
                self.datas[0].datetime.date(1),
                isbuy=True,
            )
        elif self.position and self.crossover[0] < 0:
            self.close(price=self.dataopen[1])
            self._log_tx(
                self.crossover[0],
                self.dataopen[1],
                self.datas[0].datetime.date(1),
            )
      # else:
      #   self.log(f"crossover: {self.crossover[0]}")

In [145]:
data = bt.feeds.PandasData(dataname=dataset["2019"])

# init cerebro
cerebro = bt.Cerebro()
# feed data
cerebro.adddata(data)
# add strategy
cerebro.addstrategy(SMACross)
# All In
# cerebro.addsizer(bt.sizers.AllInSizer)
# cerebro.addsizer(bt.sizers.SizerFix, stake = 1)
cerebro.addsizer(AllIn)
# add analyzer
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual_return')
# run backtest
thestrats = cerebro.run()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


2019-03-15: buy, 1, $267.07

2019-03-27: sell, -1, $267.61

2019-04-02: buy, 1, $272.30

2019-05-07: sell, -1, $276.22

...

2019-09-25: sell, -1, $284.43

2019-10-15: buy, 1, $285.52

2019-12-06: sell, -1, $301.88

2019-12-11: buy, 1, $301.79

2019-12-31: sell, 0, $309.55

In [143]:
thestrat = thestrats[0]
print("Returns:")
for k, v in thestrat.analyzers.returns.get_analysis().items():
    print(f"{k}:", f"{v * 100:.2f}%")

print("-" * 32)
print("Annual Return:")
for k, v in thestrat.analyzers.annual_return.get_analysis().items():
    print(f"{k}:", f"{v * 100:.2f}%")

Returns:

rtot: 12.39%

ravg: 0.05%

rnorm: 13.19%

rnorm100: 1318.50%

--------------------------------

Annual Return:

2019: 13.19%

In [146]:
cerebro.plot()

[[<Figure size 432x288 with 5 Axes>]]